## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-04-01-08 +0000,nyt,"No Passports, No Study Abroad: China Limits Pu...",https://www.nytimes.com/2025/08/03/world/asia/...
1,2025-08-03-03-41-15 +0000,latimes,"Gifford fire burns 30,000 acres in Los Padres ...",https://www.latimes.com/environment/story/2025...
2,2025-08-03-03-33-15 +0000,nypost,Mob of anti-Israel protestors clash with polic...,https://nypost.com/2025/08/02/us-news/mob-of-a...
3,2025-08-03-03-00-00 +0000,wsj,Swiss business officials struggle to wrap thei...,https://www.wsj.com/economy/trade/trumps-slap-...
4,2025-08-03-02-53-15 +0000,nypost,Trump tells Schumer to ‘GO TO HELL’ over Senat...,https://nypost.com/2025/08/02/us-news/trump-te...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,24
54,new,8
159,former,7
420,bar,6
129,city,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,74
17,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,48
21,2025-08-03-00-09-55 +0000,cbc,U.S. watchdog launches investigation into form...,https://www.cbc.ca/news/world/jack-smith-speci...,47
72,2025-08-02-18-38-43 +0000,nyt,Office of Special Counsel Opens Investigation ...,https://www.nytimes.com/2025/08/02/us/politics...,46
89,2025-08-02-15-31-46 +0000,nypost,Trump demurs on pardoning disgraced former Rep...,https://nypost.com/2025/08/02/us-news/presiden...,46


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,74,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
22,37,2025-08-03-00-04-44 +0000,nypost,Cincinnati police chief under intense scrutiny...,https://nypost.com/2025/08/02/us-news/cincinna...
74,30,2025-08-02-18-35-14 +0000,cbc,"U.S. has 'very, very good plan' to end war in ...",https://www.cbc.ca/news/world/us-plan-ceasefir...
98,27,2025-08-02-14-07-39 +0000,wapo,Montana bar shooting leaves 4 dead as authorit...,https://www.washingtonpost.com/nation/2025/08/...
57,27,2025-08-02-20-20-05 +0000,nypost,Bill Maher slams woke mob over Sydney Sweeney ...,https://nypost.com/2025/08/02/us-news/bill-mah...
13,26,2025-08-03-01-04-42 +0000,nypost,Virginia city Councilman Lee Vogler faces ‘lon...,https://nypost.com/2025/08/02/us-news/virginia...
65,26,2025-08-02-19-21-35 +0000,nypost,Three killed in Russia as Ukraine takes aim at...,https://nypost.com/2025/08/02/world-news/three...
118,24,2025-08-02-10-00-00 +0000,latimes,Indian businesses in SoCal brace for 25% tarif...,https://www.latimes.com/business/story/2025-08...
71,23,2025-08-02-18-39-45 +0000,nypost,Islamic Republic appoints ‘Kennedy of Iran’ to...,https://nypost.com/2025/08/02/world-news/islam...
136,22,2025-08-02-09-00-25 +0000,nyt,No Pets Left Behind: How a Dog’s Hurricane Res...,https://www.nytimes.com/2025/08/02/us/trooper-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
